In [1]:
import csv
from tqdm import tqdm
import numpy as np


raw_train_sentences = []
for i in tqdm(range(10000)):
    five_sentences = ""
    for line in open("./all/data/descriptions_train/" + str(i) + ".txt"):
        five_sentences += line.strip() + " "
    raw_train_sentences.append(five_sentences.strip())
    
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

embeddings_index = {}
f = open('glove.42B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

xtrain_glove = [sent2vec(x) for x in tqdm(raw_train_sentences)]



# Reading 2048-d train features 
features_train_2048 = np.zeros((10000,2048))
with open('./all/data/features_train/features_resnet1000intermediate_train.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[13:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_train_2048[int(image_name)] = row
        
# Reading 1000-d train features 
features_train_1000 = np.zeros((10000,1000))
with open('./all/data/features_train/features_resnet1000_train.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[13:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_train_1000[int(image_name)] = row
        
# Find set of supercategories, categories 
supercategory_set = set()
category_set = set()
for i in range (10000):
    file = open("./all/data/tags_train/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    for  line in lines:
        words = line.strip().split(':')
        supercategory_set.add(words[0])
        category_set.add(words[1])
    file.close()

# mapping from (super) category to index
supercategory_dict = {item:val for val, item in enumerate(supercategory_set)}
category_dict = {item:val+1 for val, item in enumerate(category_set)}

# Vectorize train tags
train_tags = []
for i in range (10000):
    file = open("./all/data/tags_train/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    row = np.zeros(len(supercategory_set))
    for line in lines:
        words = line.strip().split(':')
        supercategory_column = supercategory_dict.get(words[0])
        category_index = category_dict.get(words[1])
        row[supercategory_column] = category_index
    train_tags.append(row)
    file.close()

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2221.18it/s]
1917495it [03:08, 10176.74it/s]
100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1153.39it/s]


In [2]:
features_train_concat = np.concatenate((np.array(features_train_2048),np.array(features_train_1000)), axis=1)
features_train_concat = np.concatenate((features_train_concat, np.array(train_tags)), axis=1)

X_train_1 = np.concatenate((xtrain_glove,features_train_concat),axis=1)
Y_train_1 = np.ones(10000)

train_query_reordered = np.concatenate((xtrain_glove[5000:10000],xtrain_glove[0:5000]),axis=0)
X_train_0 = np.concatenate((train_query_reordered,features_train_concat),axis=1)
Y_train_0 = np.zeros(10000)

X_train = np.concatenate((X_train_1,X_train_0),axis=0)
Y_train = np.concatenate((Y_train_1,Y_train_0),axis=0)

In [9]:
from scipy import linalg as LA
from sklearn.preprocessing import StandardScaler

X_train_std = StandardScaler().fit_transform(X_train)
# mean center the data
X_train_mean = X_train_std - X_train_std.mean(axis=0)
# calculate the covariance matrix
R = np.cov(X_train_mean, rowvar=False)
# calculate eigenvectors & eigenvalues of the covariance matrix
# use 'eigh' rather than 'eig' since R is symmetric, 
# the performance gain is substantial
evals, evecs = LA.eigh(R)

In [13]:
valid = []
invalid = []
for i in range(len(evals)):
    if evals[i] > 1:
        valid.append(i)
    else:
        invalid.append(i)
        
print(len(invalid))
print(len(valid))
print(valid)

3068
292
[3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084, 3085, 3086, 3087, 3088, 3089, 3090, 3091, 3092, 3093, 3094, 3095, 3096, 3097, 3098, 3099, 3100, 3101, 3102, 3103, 3104, 3105, 3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3148, 3149, 3150, 3151, 3152, 3153, 3154, 3155, 3156, 3157, 3158, 3159, 3160, 3161, 3162, 3163, 3164, 3165, 3166, 3167, 3168, 3169, 3170, 3171, 3172, 3173, 3174, 3175, 3176, 3177, 3178, 3179, 3180, 3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197, 3198, 3199, 3200, 3201, 3202, 3203, 3204, 3205, 3206, 3207, 3208, 3209, 3210, 3211, 3212, 3213, 3214, 3215, 3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3224, 3225, 3226, 3227, 3228, 3229, 3230, 3231, 3232, 

In [12]:
print(len(evals))

3360


In [34]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

pca = PCA(n_components=292)
# Taking in as second argument the Target as labels
X_PCA_red = pca.fit_transform(X_train_std)

lda = LDA(n_components=292)
# Taking in as second argument the Target as labels
X_LDA_red = lda.fit_transform(X_train_std, Y_train)

C:\Users\Sonia\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [35]:
print(X_PCA_red.shape)
print(X_LDA_red.shape)
# from sklearn.ensemble import RandomForestClassifier
# clf_RF = RandomForestClassifier(max_depth=100, random_state=0,n_estimators=500,verbose=3,n_jobs=-1,max_features=None)
# clf_RF.fit(X_train,Y_train)

(20000, 292)
(20000, 1)


In [23]:
print(X_train.shape)
print(Y_train[-1])

(20000, 3360)
0.0


In [28]:
print(lda.Y)

AttributeError: 'PCA' object has no attribute 'Y'

In [41]:
print(lda.classes_)

[0. 1.]


In [42]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(max_depth=100, random_state=0,n_estimators=500,verbose=3,n_jobs=-1,max_features=None)
clf_RF.fit(X_PCA_red,Y_train)

building tree 1 of 500building tree 2 of 500building tree 3 of 500building tree 4 of 500building tree 5 of 500building tree 6 of 500building tree 7 of 500building tree 8 of 500







building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   25.9s



building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500building tree 32 of 500

building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 6

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  5.1min


building tree 120 of 500
building tree 121 of 500
building tree 122 of 500
building tree 123 of 500
building tree 124 of 500
building tree 125 of 500
building tree 126 of 500
building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500
building tree 141 of 500
building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500
building tree 152 of 500
building tree 153 of 500
building tree 154 of 500
building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500


[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 13.4min


building tree 280 of 500
building tree 281 of 500
building tree 282 of 500
building tree 283 of 500
building tree 284 of 500
building tree 285 of 500
building tree 286 of 500
building tree 287 of 500
building tree 288 of 500
building tree 289 of 500
building tree 290 of 500
building tree 291 of 500
building tree 292 of 500
building tree 293 of 500
building tree 294 of 500
building tree 295 of 500
building tree 296 of 500
building tree 297 of 500
building tree 298 of 500
building tree 299 of 500
building tree 300 of 500
building tree 301 of 500
building tree 302 of 500
building tree 303 of 500
building tree 304 of 500
building tree 305 of 500
building tree 306 of 500
building tree 307 of 500
building tree 308 of 500
building tree 309 of 500
building tree 310 of 500
building tree 311 of 500
building tree 312 of 500
building tree 313 of 500
building tree 314 of 500
building tree 315 of 500
building tree 316 of 500
building tree 317 of 500
building tree 318 of 500
building tree 319 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 24.2min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=0, verbose=3, warm_start=False)

In [43]:
raw_test_sentences = []
for i in tqdm(range(2000)):
    five_sentences = ""
    for line in open("./all/data/descriptions_test/" + str(i) + ".txt"):
        five_sentences += line.strip() + " "
    raw_test_sentences.append(five_sentences.strip())

xtest_glove = [sent2vec(x) for x in tqdm(raw_test_sentences)]


features_test_2048 = np.zeros((2000,2048))
with open('./all/data/features_test/features_resnet1000intermediate_test.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[12:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_test_2048[int(image_name)] = row

features_test_1000 = np.zeros((2000,1000))    
with open('./all/data/features_test/features_resnet1000_test.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[12:]
        row = []
        for i in range(len(line)):
            if i > 0:
                row.append(float(line[i]))
        features_test_1000[int(image_name)] = row

# Vectorize test tags
test_tags = []
for i in range (2000):
    file = open("./all/data/tags_test/" + str(i) + ".txt", "r")
    lines = file.readlines() 
    row = np.zeros(len(supercategory_set))
    for line in lines:
        words = line.strip().split(':')
        supercategory_column = supercategory_dict.get(words[0])
        category_index = category_dict.get(words[1])
        row[supercategory_column] = category_index
    test_tags.append(row)
    file.close()
    
features_test_concat = np.concatenate((np.array(features_test_2048),np.array(features_test_1000)), axis=1)
features_test_concat = np.concatenate((features_test_concat, np.array(test_tags)), axis=1)





# rank_list = []
# for i in range (2000):
#     query_repeat = np.tile(xtest_glove[i],(2000,1))
#     X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
#     svm_predictions = clf_RF.predict_proba(X_test)
#     predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
#     predictions_indices.sort(reverse=True)
#     rank_list.append([v for (k,v) in predictions_indices[:20]])
# def mergeTwenty(lst):
#     result = str(lst[0])+".jpg"
#     for i in range(1,len(lst)):
#         result += " " + str(lst[i]) + ".jpg"
#     return result

# with open('RF_full_data_submission.csv', 'w', newline='') as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=',')
#     csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
#     for i in range(len(rank_list)):
#         csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 828.99it/s]


In [44]:
clf_RF.predict_proba(X_PCA_red[:1])

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished


array([[0.186, 0.814]])

In [45]:
rank_list = []
for i in tqdm(range (2000)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = pca.transform(X_test)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('RF_full_data_submission.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

  0%|                                                                                         | 0/2000 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  0%|                                                                                 | 1/2000 [00:00<11:33,  2.88it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  0%|                                                                                 | 2/2000 [00:00<11:24,  2.92it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

  1%|▉                                                                               | 22/2000 [00:08<11:19,  2.91it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  1%|▉                                                                               | 23/2000 [00:08<11:40,  2.82it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  1%|▉                                                                               | 24/2000 [00:08<12:45,  2.58it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

  2%|█▊                                                                              | 44/2000 [00:16<12:00,  2.72it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  2%|█▊                                                                              | 45/2000 [00:16<11:43,  2.78it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  2%|█▊                                                                              | 46/2000 [00:17<11:40,  2.79it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

  3%|██▋                                                                             | 66/2000 [00:24<11:21,  2.84it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  3%|██▋                                                                             | 67/2000 [00:24<11:23,  2.83it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  3%|██▋                                                                             | 68/2000 [00:25<11:31,  2.79it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

  4%|███▌                                                                            | 88/2000 [00:33<13:03,  2.44it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  4%|███▌                                                                            | 89/2000 [00:33<12:21,  2.58it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
  4%|███▌                                                                            | 90/2000 [00:33<11:49,  2.69it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

...

 18%|█████████████▉                                                                 | 352/2000 [02:25<15:31,  1.77it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished
 18%|█████████████▉                                                                 | 353/2000 [02:26<15:48,  1.74it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.3s finished
 18%|█████████████▉                                                                 | 354/2000 [02:27<16:56,  1.62it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

 19%|██████████████▊                                                                | 374/2000 [02:38<14:34,  1.86it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 19%|██████████████▊                                                                | 375/2000 [02:39<14:05,  1.92it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 19%|██████████████▊                                                                | 376/2000 [02:39<13:42,  1.98it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

 20%|███████████████▋                                                               | 396/2000 [02:50<14:18,  1.87it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 20%|███████████████▋                                                               | 397/2000 [02:50<15:01,  1.78it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 20%|███████████████▋                                                               | 398/2000 [02:51<15:24,  1.73it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

 21%|████████████████▌                                                              | 418/2000 [03:01<12:48,  2.06it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 21%|████████████████▌                                                              | 419/2000 [03:02<12:41,  2.08it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished
 21%|████████████████▌                                                              | 420/2000 [03:02<12:48,  2.06it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_job

KeyboardInterrupt: 

In [46]:
rank_list = []
for i in tqdm(range (2000)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_red = pca.transform(X_test)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  0%|                                                                                 | 1/2000 [00:00<11:45,  2.83it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  0%|                                                                                 | 2/2000 [00:00<11:32,  2.89it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


  1%|▉                                                                               | 22/2000 [00:08<12:14,  2.69it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  1%|▉                                                                               | 23/2000 [00:08<12:00,  2.74it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  1%|▉                                                                               | 24/2000 [00:08<11:58,  2.75it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


  2%|█▊                                                                              | 44/2000 [00:16<11:48,  2.76it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  2%|█▊                                                                              | 45/2000 [00:17<11:41,  2.79it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  2%|█▊                                                                              | 46/2000 [00:17<11:34,  2.81it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


  3%|██▋                                                                             | 66/2000 [00:25<12:07,  2.66it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  3%|██▋                                                                             | 67/2000 [00:25<11:49,  2.72it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  3%|██▋                                                                             | 68/2000 [00:26<11:30,  2.80it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


  4%|███▌                                                                            | 88/2000 [00:33<11:15,  2.83it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  4%|███▌                                                                            | 89/2000 [00:33<11:13,  2.84it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

  4%|███▌                                                                            | 90/2000 [00:33<11:06,  2.86it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_

...


 95%|█████████████████████████████████████████████████████████████████████████▊    | 1892/2000 [17:10<00:58,  1.83it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 95%|█████████████████████████████████████████████████████████████████████████▊    | 1893/2000 [17:11<00:57,  1.85it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 95%|█████████████████████████████████████████████████████████████████████████▊    | 1894/2000 [17:11<00:56,  1.89it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


 96%|██████████████████████████████████████████████████████████████████████████▋   | 1914/2000 [17:22<00:45,  1.87it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 96%|██████████████████████████████████████████████████████████████████████████▋   | 1915/2000 [17:22<00:44,  1.90it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 96%|██████████████████████████████████████████████████████████████████████████▋   | 1916/2000 [17:23<00:43,  1.92it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


 97%|███████████████████████████████████████████████████████████████████████████▌  | 1936/2000 [17:34<00:35,  1.79it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 97%|███████████████████████████████████████████████████████████████████████████▌  | 1937/2000 [17:34<00:34,  1.85it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 97%|███████████████████████████████████████████████████████████████████████████▌  | 1938/2000 [17:35<00:32,  1.88it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


 98%|████████████████████████████████████████████████████████████████████████████▎ | 1958/2000 [17:45<00:21,  1.91it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 98%|████████████████████████████████████████████████████████████████████████████▍ | 1959/2000 [17:46<00:21,  1.92it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 98%|████████████████████████████████████████████████████████████████████████████▍ | 1960/2000 [17:46<00:20,  1.94it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_


 99%|█████████████████████████████████████████████████████████████████████████████▏| 1980/2000 [17:58<00:10,  1.94it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 99%|█████████████████████████████████████████████████████████████████████████████▎| 1981/2000 [17:58<00:09,  1.99it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.2s finished

 99%|█████████████████████████████████████████████████████████████████████████████▎| 1982/2000 [17:59<00:09,  1.98it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_

In [47]:
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('RF_full_data_submission.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])

In [48]:
for i in tqdm(range (2)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
    rank_list.append([v for (k,v) in predictions_indices[:20]])


  0%|                                                                                            | 0/2 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  1.71it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.73it/s]


In [50]:
print(rank_list[-1])
print(rank_list[-2])

[1998, 951, 760, 670, 1466, 1798, 533, 1430, 782, 1498, 1026, 696, 1971, 1510, 959, 784, 1400, 1942, 584, 564]
[1998, 951, 760, 670, 1466, 1798, 533, 1430, 782, 1498, 1026, 696, 1971, 1510, 959, 784, 1400, 1942, 584, 564]


In [51]:
print(len(rank_list))

2002


In [52]:
print(len(rank_list[-1]))

20


In [59]:
A = clf_RF.predict_proba(pca.transform(StandardScaler().fit_transform(np.concatenate((xtrain_glove[:1],features_train_concat[:1]),axis=1))))
print(A)

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished


[[0.012 0.988]]


In [60]:
print(X_PCA_red[:1])

[[-9.96232606e+00 -1.07162576e+01  7.47656428e+00 -4.55482467e+00
   7.71930911e+00 -3.89471551e+00  7.26946245e+00  1.09591198e+01
   3.28574744e+00 -7.10985661e-01 -4.66988963e+00  9.18587693e-01
   4.20991981e+00 -5.89394593e+00  2.94160020e+00 -2.87474133e+00
   6.60699482e+00  2.57742982e+00 -8.83150089e-01  1.54839837e+00
   7.75064879e-01 -1.23173943e+01 -1.97650643e+00 -1.78593812e+00
   6.09295673e+00  1.73753712e+00 -2.60207559e+00 -4.78151673e-01
   5.82961496e+00  1.18995030e+00  3.94830034e+00 -6.18346846e+00
  -8.61069242e-01  5.98715892e+00  3.09901106e+00 -1.06038538e+00
   3.19901934e+00 -6.91813718e+00  2.69445474e-01 -5.98106879e+00
   3.19320198e+00  8.60959726e+00  2.81505917e+00  4.78242106e+00
   3.14833611e+00 -4.04899736e+00 -7.55207794e-01  3.21601020e+00
   2.21242597e+00 -1.23999343e+00  5.20429508e-01 -2.10061723e+00
  -7.98336354e-01 -4.99770696e+00  3.86911570e+00 -6.72273637e+00
  -1.11499647e-01  1.68158197e+00  8.61448018e-01  3.26112254e+00
   2.26351

In [61]:
print(pca.transform(StandardScaler().fit_transform(np.concatenate((xtrain_glove[:1],features_train_concat[:1]),axis=1))))

[[ 6.04205723e-17 -5.18248888e-17 -1.06638591e-16  4.36915681e-17
  -2.79004931e-17 -9.52084818e-17  5.36376597e-17  5.94011338e-17
  -4.23411884e-18 -1.83938604e-17 -8.58751016e-17  2.11991856e-17
  -5.12725309e-17  9.23660827e-17 -6.24901936e-17  9.06092031e-17
  -5.43431333e-18 -3.08082957e-17  8.38875333e-17 -8.30461542e-17
   1.50352788e-17 -1.16101584e-16  7.42205280e-17 -4.51543934e-17
  -6.40913646e-17 -1.19203875e-16 -3.50308443e-17 -3.82319960e-17
  -2.54502804e-17 -1.00693511e-16  7.64523720e-18  7.74532833e-17
  -2.97564477e-17 -7.21428419e-17  1.61567089e-17 -3.91530862e-17
   1.57523841e-17  1.22444382e-16 -1.47208291e-17  5.49422995e-18
   7.97915228e-17 -1.11786533e-16  1.46835903e-18  1.39467016e-16
  -1.14443594e-16  2.21306443e-18 -3.35454118e-17 -1.82232034e-17
  -2.93716511e-18 -1.75760125e-17  5.26954886e-17  1.18664838e-18
  -5.97185947e-17  2.47138093e-17 -2.74800517e-17 -1.34345694e-16
  -8.85616155e-17 -1.64935378e-17  5.86451897e-17 -1.27325718e-17
   1.80584

In [62]:
for i in tqdm(range (1)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
print(predictions_indices)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


[(0.874, 1998), (0.864, 951), (0.862, 760), (0.862, 670), (0.856, 1466), (0.844, 1798), (0.832, 533), (0.826, 1430), (0.826, 782), (0.824, 1498), (0.824, 1026), (0.824, 696), (0.822, 1971), (0.822, 1510), (0.822, 959), (0.822, 784), (0.82, 1400), (0.818, 1942), (0.818, 584), (0.818, 564), (0.816, 1465), (0.81, 669), (0.808, 1332), (0.806, 1011), (0.806, 764), (0.804, 1594), (0.802, 1231), (0.802, 914), (0.8, 1451), (0.798, 1501), (0.798, 472), (0.796, 1700), (0.796, 1362), (0.796, 1302), (0.796, 1123), (0.796, 805), (0.796, 462), (0.794, 1154), (0.79, 1912), (0.788, 207), (0.786, 1822), (0.786, 1132), (0.786, 338), (0.782, 1633), (0.782, 1321), (0.78, 624), (0.778, 617), (0.776, 638), (0.776, 525), (0.774, 1215), (0.77, 1219), (0.77, 155), (0.768, 1985), (0.768, 1453), (0.768, 1055), (0.768, 68), (0.766, 978), (0.764, 1564), (0.764, 305), (0.762, 1218), (0.762, 956), (0.762, 791), (0.762, 552), (0.762, 235), (0.76, 960), (0.76, 834), (0.76, 691), (0.76, 677), (0.758, 1228), (0.758, 52)

In [63]:
for i in tqdm(range (1,2)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
print(predictions_indices)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.76it/s]


[(0.874, 1998), (0.864, 951), (0.862, 760), (0.862, 670), (0.856, 1466), (0.844, 1798), (0.832, 533), (0.826, 1430), (0.826, 782), (0.824, 1498), (0.824, 1026), (0.824, 696), (0.822, 1971), (0.822, 1510), (0.822, 959), (0.822, 784), (0.82, 1400), (0.818, 1942), (0.818, 584), (0.818, 564), (0.816, 1465), (0.81, 669), (0.808, 1332), (0.806, 1011), (0.806, 764), (0.804, 1594), (0.802, 1231), (0.802, 914), (0.8, 1451), (0.798, 1501), (0.798, 472), (0.796, 1700), (0.796, 1362), (0.796, 1302), (0.796, 1123), (0.796, 805), (0.796, 462), (0.794, 1154), (0.79, 1912), (0.788, 207), (0.786, 1822), (0.786, 1132), (0.786, 338), (0.782, 1633), (0.782, 1321), (0.78, 624), (0.778, 617), (0.776, 638), (0.776, 525), (0.774, 1215), (0.77, 1219), (0.77, 155), (0.768, 1985), (0.768, 1453), (0.768, 1055), (0.768, 68), (0.766, 978), (0.764, 1564), (0.764, 305), (0.762, 1218), (0.762, 956), (0.762, 791), (0.762, 552), (0.762, 235), (0.76, 960), (0.76, 834), (0.76, 691), (0.76, 677), (0.758, 1228), (0.758, 52)

In [66]:
clf_RF.predict_proba(X_PCA_red[:3])

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.0s finished


array([[0.186, 0.814],
       [0.098, 0.902],
       [0.13 , 0.87 ]])

In [67]:
for i in tqdm(range (2,3)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)
print(predictions_indices)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.72it/s]


[(0.874, 1998), (0.864, 951), (0.862, 760), (0.862, 670), (0.856, 1466), (0.844, 1798), (0.832, 533), (0.826, 1430), (0.826, 782), (0.824, 1498), (0.824, 1026), (0.824, 696), (0.822, 1971), (0.822, 1510), (0.822, 959), (0.822, 784), (0.82, 1400), (0.818, 1942), (0.818, 584), (0.818, 564), (0.816, 1465), (0.81, 669), (0.808, 1332), (0.806, 1011), (0.806, 764), (0.804, 1594), (0.802, 1231), (0.802, 914), (0.8, 1451), (0.798, 1501), (0.798, 472), (0.796, 1700), (0.796, 1362), (0.796, 1302), (0.796, 1123), (0.796, 805), (0.796, 462), (0.794, 1154), (0.79, 1912), (0.788, 207), (0.786, 1822), (0.786, 1132), (0.786, 338), (0.782, 1633), (0.782, 1321), (0.78, 624), (0.778, 617), (0.776, 638), (0.776, 525), (0.774, 1215), (0.77, 1219), (0.77, 155), (0.768, 1985), (0.768, 1453), (0.768, 1055), (0.768, 68), (0.766, 978), (0.764, 1564), (0.764, 305), (0.762, 1218), (0.762, 956), (0.762, 791), (0.762, 552), (0.762, 235), (0.76, 960), (0.76, 834), (0.76, 691), (0.76, 677), (0.758, 1228), (0.758, 52)

In [69]:
for i in tqdm(range (1)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    print(X_test_red)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[[ 1.04355561e+01 -1.31007364e+01 -6.69185382e-01 ...  7.13939962e-01
   5.52287511e-01  1.19121460e+00]
 [-1.28239560e+01  1.50088847e+01  8.09448832e+00 ... -6.62138197e-02
  -3.76892693e-01  8.14422058e-02]
 [ 8.86196199e+00 -1.10716090e+01  1.75253472e+01 ... -5.17266793e-01
   6.58357956e-01 -3.85210940e-01]
 ...
 [-1.76064306e+01 -5.52144923e-01 -1.30561149e+01 ... -8.22685810e-01
   1.38863266e+00 -2.28191997e-01]
 [-1.70093016e+01  2.45860705e+00 -9.12149345e+00 ... -2.40899638e-01
  -6.14438717e-01 -1.64087297e-02]
 [ 2.35744225e+01  3.15234872e+01 -4.12201663e+00 ...  1.23580754e-01
  -1.14930827e-01 -1.66214642e-01]]


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.66it/s]


In [71]:
for i in tqdm(range (1,2)):
    query_repeat = np.tile(xtest_glove[i],(2000,1))
    X_test = np.concatenate((query_repeat,features_test_concat),axis=1)
    X_test_std = StandardScaler().fit_transform(X_test)
    X_test_red = pca.transform(X_test_std)
    print(X_test_red)
    svm_predictions = clf_RF.predict_proba(X_test_red)
    predictions_indices = [(svm_predictions[index][1],index) for index in range(2000)]
    predictions_indices.sort(reverse=True)



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

[[ 1.04355561e+01 -1.31007364e+01 -6.69185382e-01 ...  7.13939962e-01
   5.52287511e-01  1.19121460e+00]
 [-1.28239560e+01  1.50088847e+01  8.09448832e+00 ... -6.62138197e-02
  -3.76892693e-01  8.14422058e-02]
 [ 8.86196199e+00 -1.10716090e+01  1.75253472e+01 ... -5.17266793e-01
   6.58357956e-01 -3.85210940e-01]
 ...
 [-1.76064306e+01 -5.52144923e-01 -1.30561149e+01 ... -8.22685810e-01
   1.38863266e+00 -2.28191997e-01]
 [-1.70093016e+01  2.45860705e+00 -9.12149345e+00 ... -2.40899638e-01
  -6.14438717e-01 -1.64087297e-02]
 [ 2.35744225e+01  3.15234872e+01 -4.12201663e+00 ...  1.23580754e-01
  -1.14930827e-01 -1.66214642e-01]]


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.51it/s]
